In [1]:
import json
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import numpy as np
import pandas as pd
import operator
import seaborn as sns
import math
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import imblearn.over_sampling as imb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score

In [2]:
nations = ['Italy','England','France','Spain','European_Championship','World_Cup']

events = {}
events['England'] = pd.read_json('/Users/lucastyler/Downloads/Senior Fall/DSCI 303/Project DSCI 303/events/events_England.json')
events['Italy'] = pd.read_json('/Users/lucastyler/Downloads/Senior Fall/DSCI 303/Project DSCI 303/events/events_Italy.json')
events['France'] = pd.read_json('/Users/lucastyler/Downloads/Senior Fall/DSCI 303/Project DSCI 303/events/events_France.json')
events['Spain'] = pd.read_json('/Users/lucastyler/Downloads/Senior Fall/DSCI 303/Project DSCI 303/events/events_Spain.json')
events['European_Championship'] = pd.read_json('/Users/lucastyler/Downloads/Senior Fall/DSCI 303/Project DSCI 303/events/events_European_Championship.json')
events['World_Cup'] = pd.read_json('/Users/lucastyler/Downloads/Senior Fall/DSCI 303/Project DSCI 303/events/events_World_Cup.json')

matches = {}
matches['England'] = pd.read_json('/Users/lucastyler/Downloads/Senior Fall/DSCI 303/Project DSCI 303/matches/matches_England.json')
matches['Italy'] = pd.read_json('/Users/lucastyler/Downloads/Senior Fall/DSCI 303/Project DSCI 303/matches/matches_Italy.json')
matches['France'] = pd.read_json('/Users/lucastyler/Downloads/Senior Fall/DSCI 303/Project DSCI 303/matches/matches_France.json')
matches['Spain'] = pd.read_json('/Users/lucastyler/Downloads/Senior Fall/DSCI 303/Project DSCI 303/matches/matches_Spain.json')
matches['European_Championship'] = pd.read_json('/Users/lucastyler/Downloads/Senior Fall/DSCI 303/Project DSCI 303/matches/matches_European_Championship.json')
matches['World_Cup'] = pd.read_json('/Users/lucastyler/Downloads/Senior Fall/DSCI 303/Project DSCI 303/matches/matches_World_Cup.json')

teams = pd.read_json('/Users/lucastyler/Downloads/Senior Fall/DSCI 303/Project DSCI 303/teams.json')

stadiums = pd.read_excel('/Users/lucastyler/Downloads/Senior Fall/DSCI 303/Project DSCI 303/Stadium field dimensions.xlsx')


In [3]:
Englandshots = events['England']['eventName'] == 'Shot'
shots = events['England'][Englandshots]
all_matches = matches['England']
nationsnoEng = ['Italy','France','Spain','European_Championship','World_Cup']

for nation in nationsnoEng:
    all_matches = pd.concat([all_matches, matches[nation]])
    shotsbool = events[nation]['eventName'] == 'Shot'
    onlyshots = events[nation][shotsbool]
    shots = pd.concat([shots, onlyshots])

In [4]:
shots['x'] = [x[0]['x'] for x in shots['positions']]
shots['y'] = [x[0]['y'] for x in shots['positions']]

In [5]:
players = pd.read_json('/Users/lucastyler/Downloads/Senior Fall/DSCI 303/Project DSCI 303/players.json')

In [6]:
players['position'] = [x['name'] for x in players['role']]

In [7]:
playersprac = players[['firstName', 'lastName', 'height', 'weight', 'shortName', 'foot', 'wyId', 'position']]
shots = pd.merge(shots, playersprac, left_on = 'playerId', right_on = 'wyId')
shots

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,...,x,y,firstName,lastName,height,weight,shortName,foot,wyId,position
0,10,Shot,"[{'id': 101}, {'id': 402}, {'id': 201}, {'id':...",25413,"[{'y': 41, 'x': 88}, {'y': 0, 'x': 0}]",2499719,Shot,1609,1H,94.595788,...,88,41,Alexandre,Lacazette,175,73,A. Lacazette,right,25413,Forward
1,10,Shot,"[{'id': 402}, {'id': 2101}, {'id': 1802}]",25413,"[{'y': 48, 'x': 87}, {'y': 0, 'x': 0}]",2499719,Shot,1609,1H,2813.844402,...,87,48,Alexandre,Lacazette,175,73,A. Lacazette,right,25413,Forward
2,10,Shot,"[{'id': 401}, {'id': 201}, {'id': 1207}, {'id'...",25413,"[{'y': 34, 'x': 90}, {'y': 0, 'x': 0}]",2499719,Shot,1609,2H,2354.807116,...,90,34,Alexandre,Lacazette,175,73,A. Lacazette,right,25413,Forward
3,10,Shot,"[{'id': 402}, {'id': 201}, {'id': 1212}, {'id'...",25413,"[{'y': 33, 'x': 90}, {'y': 100, 'x': 100}]",2499743,Shot,1609,2H,1404.650587,...,90,33,Alexandre,Lacazette,175,73,A. Lacazette,right,25413,Forward
4,10,Shot,"[{'id': 101}, {'id': 402}, {'id': 201}, {'id':...",25413,"[{'y': 58, 'x': 82}, {'y': 0, 'x': 0}]",2499749,Shot,1609,1H,1616.714627,...,82,58,Alexandre,Lacazette,175,73,A. Lacazette,right,25413,Forward
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36172,10,Shot,"[{'id': 402}, {'id': 201}, {'id': 1216}, {'id'...",209091,"[{'y': 41, 'x': 80}, {'y': 100, 'x': 100}]",2058010,Shot,4418,2H,1637.488113,...,80,41,Corentin,Tolisso,181,78,C. Tolisso,right,209091,Midfielder
36173,10,Shot,"[{'id': 401}, {'id': 201}, {'id': 1202}, {'id'...",209091,"[{'y': 35, 'x': 87}, {'y': 0, 'x': 0}]",2058014,Shot,4418,2H,3016.680937,...,87,35,Corentin,Tolisso,181,78,C. Tolisso,right,209091,Midfielder
36174,10,Shot,"[{'id': 403}, {'id': 201}, {'id': 1215}, {'id'...",101953,"[{'y': 57, 'x': 92}, {'y': 0, 'x': 0}]",2058012,Shot,14358,2H,1590.119521,...,92,57,Aleksandr,Erokhin,195,79,A. Erokhin,right,101953,Midfielder
36175,10,Shot,"[{'id': 403}, {'id': 201}, {'id': 1213}, {'id'...",101953,"[{'y': 39, 'x': 94}, {'y': 0, 'x': 0}]",2058012,Shot,14358,E1,844.590203,...,94,39,Aleksandr,Erokhin,195,79,A. Erokhin,right,101953,Midfielder


In [8]:
shots = shots[shots['position'] != 'Goalkeeper']

In [ ]:
all_matches['Both_Teams'] = [tuple(x.keys()) for x in all_matches['teamsData']]

all_matches['Team1'] = [x[0] for x in all_matches['Both_Teams']]
all_matches['Team2'] = [x[1] for x in all_matches['Both_Teams']]

shots = pd.merge(shots, all_matches, left_on = 'matchId', right_on = 'wyId')

In [ ]:
#len(shots[~shots['venue'].isin(stadiums['Stadium'].unique())])
#To add in Default stadium length of 105*68 for those lost in the merge
#Need to make sure it reads Stadia with an accent correctly
#shots['venue_2'] = ['Default' if x not in stadiums['Stadium'].unique() else x for x in shots['venue']]

In [ ]:
shots = pd.merge(shots, stadiums, left_on = 'venue', right_on = 'Stadium')
shots

In [ ]:
#probably change this variable to distance_perc
shots['distancenoadj'] = np.sqrt(np.power(shots["x"] - 100, 2) + np.power(shots["y"] - 50, 2))
#probably change this variable to distance_m
shots['adjusteddistance'] = np.sqrt(np.power((0.01*shots["x"]*shots["Length"]) 
                                                  - shots["Length"], 2) + 
                                         np.power((0.01*shots["y"]*shots["Width"]) 
                                                  - (0.5*shots["Width"]), 2))

In [9]:
tagvalscol = []
for x in shots['tags']:
    tagvals = []
    for i in range(len(x)):
        tagvals.append(x[i]['id'])
    tagvalscol.append(tagvals)
shots['tagvals'] = tagvalscol

<ipython-input-9-8065e7bd3c59>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shots['tagvals'] = tagvalscol


In [12]:
shots['goal'] = [1 if 101 in x else 0 for x in shots['tagvals']]

<ipython-input-12-10aa6d188ce7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shots['goal'] = [1 if 101 in x else 0 for x in shots['tagvals']]


In [ ]:
shots.hist('goal')

In [ ]:
df_corr = shots.corr()
plt.figure(figsize=(24, 24))

sns.heatmap(df_corr, 
            cmap='viridis', vmax=1.0, vmin=-1.0, linewidths=0.1,
            annot=True, annot_kws={"size": 8}, square=True);

In [ ]:
shots['foot_shot'] = ['right' if 402 in x else 'left' if 401 in x else 'head/body' for x in shots['tagvals']]

In [ ]:
#procedure for creating dummy variables is the same as in the below column, can actually add it into the same for loop
#that being said, would there be an issue for adding in only two observations with goalkeeper shots where one is a
#goal? Might the algorithm think its extremely likely for goalkeepers to score when they're shooting when in fact this
#is just because of how few observations we have of goalkeepers shooting? Because of course we assume Forwards are
#most likely to score. But also, is this largely just because they shoot from the most advanced positions, at least
#from the view of the dataset?
positions = ['Forward', 'Midfielder', 'Defender']
for pos in positions:
    pos_shots = shots[shots['position'] == pos]
    goalsfrompos = pos_shots[pos_shots['goal'] == 1]
    perc_goals = len(goalsfrompos)/len(pos_shots)
    print('The percentage of successful shots from the', pos, 'position in this database is: ', perc_goals)

In [ ]:
#1 means the player shot with their strong foot, 0 means they shot with their weak foot or
#with their head/body
strong_foot = []
header = []
for idx, row in shots.iterrows():
    if row['foot_shot'] == row['foot']:
        strong_foot.append(1)
    else:
        strong_foot.append(0)
    if row['foot_shot'] == 'head/body':
        header.append(1)
    else:
        header.append(0)

In [ ]:
forward = []
midfielder = []
defender = []
for idx, row in shots.iterrows():
    if row['position'] == 'Forward':
        forward.append(1)
        midfielder.append(0)
        defender.append(0)
    elif row['position'] == 'Midfielder':
        forward.append(0)
        midfielder.append(1)
        defender.append(0)
    else:
        forward.append(0)
        midfielder.append(0)
        defender.append(1)

In [ ]:
shots['F'] = forward
shots['MF'] = midfielder
shots['DEF'] = defender

In [ ]:
shots['strong_foot'] = strong_foot
shots['header'] = header

In [ ]:
#Add 45*60 seconds to every event in the second half, as eventSec tracks the number of seconds since
#the start of the current half
eventTime = []
for idx, row in shots.iterrows():
    if row['matchPeriod'] == '2H':
        eventTime.append(row['eventSec'] + (45*60))
    else:
        eventTime.append(row['eventSec'])

In [ ]:
shots['eventTime'] = eventTime

In [ ]:
#add a column for the shot angle
#Looked at a few stadiums in Italy and they all had dimensions of 105*68m
angles = []
for idx, row in shots.iterrows():
    #If shooting from touchline, angle will be zero
    #continue to avoid divide by zero errors
    if row['x'] == 100:
        angles.append(0)
        continue
    #moving everything to left side of the field to make calculation easier
    if row['y'] > 50:
        mirror = 100 - row['y']
    else:
        mirror = row['y']
    #following four lines attempt to adjust to the actual field width and get 
    #more accurate positioning of the goal posts. e.g. Previously had right goal post at 
    #53.6 as opposed to most often at 54.64 which implicitly assumed width was 100 (which
    # is never the case). Thus, divide width by goal length to get percentage of field it takes up
    #and add half that percentage evenly to either side of 50
    goal_perc = (7.32/row['Width'])*100
    increment = goal_perc/2
    left_post = 50 - increment
    right_post = 50 + increment
    dist_a = ((row['x'] - 100)**(2))**(1/2)
    dist_b = ((mirror - right_post)**2)**(1/2)
    dist_c = ((mirror - left_post)**2)**(1/2)
    theta = np.arctan(dist_b/dist_a)*(180/math.pi)
    phi = np.arctan(dist_c/dist_a)*(180/math.pi)
    #Different cases for between the goalposts vs. to the left of a goalpost
    if mirror < left_post:
        angles.append(theta - phi)
    else:
        angles.append(theta + phi)
shots['angle'] = angles

In [ ]:
shots[['x', 'y', 'angle']]

In [ ]:
y = shots['goal']
#weakfoot is base case for what foot the shot is, defender is base case for position group
X = shots[['eventTime', 'adjusteddistance', 'angle', 'strong_foot', 'header']]

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
#should split before oversampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [ ]:
# transform the dataset
oversample = imb.SMOTE(sampling_strategy = (1/2))
X_over, y_over = oversample.fit_resample(X_train, y_train)

In [ ]:
#naive oversampling
oversample2 = imb.RandomOverSampler(sampling_strategy=0.5)
X_over2, y_over2 = oversample2.fit_resample(X, y)

In [ ]:
len(y_over2[y_over2 == 1])/len(y_over2)

In [ ]:
len(y_over[y_over == 1])/len(y_over)

In [ ]:
#Model fitted on untransformed dataset with apparently very low predictive power
xGmodel2 = LogisticRegression(random_state = 0).fit(X_train, y_train)
xGmodel2.coef_

In [ ]:
#This is the model fitted on the transformed dataset
xGmodel = LogisticRegression(random_state=0).fit(X_over, y_over)

In [ ]:
#see if I can use one of their earlier versions of the code to see how the coefficients have changed
#angle effect is much smaller by now
print(xGmodel.coef_)

In [ ]:
y_pred_train = xGmodel.predict(X_train)
y_pred_test = xGmodel.predict(X_test)

In [ ]:
print(len(y_pred_test))
print(len(y_pred_test[y_pred_test == 1]))

In [ ]:
goalprobs = [x[1] for x in xGmodel.predict_proba(X_test)]
goalprobs = np.asarray(goalprobs)
len(goalprobs[goalprobs > 0.5])
#Classification such that if the model spits out a probability of scoring greater than 0.5, will be a goal

In [ ]:
print('Accuracy of logistic regression classifier on training set: {:.2f}'.format(xGmodel.score(X_train, y_train)))
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(xGmodel.score(X_test, y_test)))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred_test)
print(confusion_matrix)

In [ ]:
#Reason for this confusion matrix--high accuracy but low recall and f1-score
num_goals = len(shots[shots['goal'] == 1])
num_misses = len(shots[shots['goal'] == 0])
perc_goals = num_goals/len(shots)
perc_misses = num_misses/len(shots)
print('Percentage of goals is: ', perc_goals)
print('Number of goals is', num_goals, ', number of misses is', num_misses)
#Attempt to use SMOTE and undersampling to rectify this
#SMOTE rather than normal oversampling

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_test))

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, xGmodel.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, xGmodel.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

In [ ]:
#website for us to compare xG results with: https://understat.com/league/EPL/2017

In [ ]:
clf = RandomForestClassifier(random_state=0, oob_score = True, max_depth = 5, n_estimators = 500)
clf.fit(X_over, y_over)

In [ ]:
y_pred_forest_train = clf.predict(X_train)
y_pred_forest_test = clf.predict(X_test)

In [ ]:
#X = shots[['eventTime', 'adjusteddistance', 'angle','strong_foot', 'header']]
clf.feature_importances_

In [ ]:
print('Accuracy of random forest classifier on training set: {:.2f}'.format(clf.score(X_train, y_train)))
print('Accuracy of random forest classifier on test set: {:.2f}'.format(clf.score(X_test, y_test)))
print('OOB score of random forest classifier on training set: {:.2f}'.format(clf.oob_score_))
print('OOB score of random forest classifier on test set: {:.2f}'.format(clf.oob_score_))

In [ ]:
print(classification_report(y_test, y_pred_forest_test))

In [ ]:
#can give the probability any shot ends up in a goal for our dataset as the predicted probability of a 1
#instance
shots['xGSMOTE'] = [x[1] for x in xGmodel.predict_proba(X)]
shots['xGnonSMOTE'] = [x[1] for x in xGmodel2.predict_proba(X)]

In [ ]:
#This says Arsenal's xG for the season is around 164 which is basically double what it actually was
#Our model is grossly overpredicting when using SMOTE
#The sum of xG seems to make a lot more sense without SMOTE, especially when taking into account that there seems
#to be missing values in the dataset
print(sum(shots[shots['teamId'] == 1609]['xGSMOTE']))
print(sum(shots[shots['teamId'] == 1609]['xGnonSMOTE']))

In [ ]:
#It seems that we are missing some goals for Arsenal, as they actually scored 74 goals in this season
#But I really think I parsed the tags correctly??
#They must have been merged out when we lost all of that data with the stadium names, although that doesnt make too
#much sense because there are still 38 games present for Arsenal
#In order for this analysis I should also merge the teams dataset
#I genuinely think its just missing some data points, which is really too bad
Arsenalshots = shots[shots['teamId'] == 1609]
len(Arsenalshots[Arsenalshots['goal'] == 1])

In [ ]:
#Just readd this one in later
shotswteam = pd.merge(shots, teams, left_on = 'teamId', right_on = 'wyId')